<a href="https://colab.research.google.com/github/yenchunR/NLP_HW4/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Model

In [27]:
!pip3 install hanziconv

     |████████████████████████████████| 276 kB 5.1 MB/s 
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23225 sha256=51fb4f72de77b86b5812f5b76df9edda70e2e32c2f2d7d83c557d9b6e1a88ba5
  Stored in directory: /root/.cache/pip/wheels/bf/e3/22/7bf50146a3ee95d1fdcbfabc44a1fe15b6e2ab7348ab7337bf
Successfully built hanziconv


In [91]:
!pip3 install gensim

# 將Colab連結Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 讀取所有檔案路徑

In [3]:
import pandas as pd
import os

# 把列出資料夾的程式碼寫成一個函式
filePath = []
def show_folder_content(folder_path):

    folder_content = os.listdir(folder_path)
    for item in folder_content:
        if os.path.isdir(folder_path + '/' + item):
            # 呼叫自己處理這個子資料夾
            show_folder_content(folder_path + '/' + item)
        elif os.path.isfile(folder_path + '/' + item):
            filePath.append(folder_path + '/' + item)
        else:
            print('無法辨識：' + item)


# 顯示指定資料夾的內容
target_folder = '/content/drive/My Drive/wiki_zh'
show_folder_content(target_folder)
print("Finish", len(filePath))

Finish 1280


# jieba 斷詞

In [4]:
# coding=utf-8
import jieba 
import json
import re

# 讀取停用詞
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

def LoadToSeg(filepath, fW):
    with open(filePath[0], 'r') as obj:
        for line in obj.readlines():
            dic = json.loads(line)
            temp_text = re.findall('[\u4e00-\u9fa5]', dic['text'])
            text = ''.join(temp_text)
            tags = jieba.lcut(text, cut_all=False)
            ThisLine = ''
            for tag in tags:
                if tag not in stopwords:  
                    if tag != '\t' and tag != '\n' and tag != '' and len(tag) > 1:  
                        ThisLine += tag  
                        ThisLine += " "   #再次組合成【帶空格】的串
            fW.write((ThisLine+"\n").encode('utf-8'))
            
stopwords = stopwordslist('/content/drive/My Drive/wiki_zh/DriveUploader/cn_stopwords.txt')  # 這裏加載停用詞的路徑 
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    fW.truncate(0)
    index = 0
    for path in filePath:
        LoadToSeg(path, fW)
        index += 1
        if index % 100 == 0:
            print(index)

print("Finish")

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.013 seconds.
Prefix dict has been built successfully.


Finish


# word2vec 轉成高維空間向量

In [113]:
from gensim.models import word2vec
# jieba分詞轉word2vec向量
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.txt'
fileSegWordDonePath2 ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.model'

sentences = word2vec.LineSentence(fileSegWordDonePath)
model = word2vec.Word2Vec(sentences, size=250, min_count=5, negative=5, workers=3, sg=1, verbose = True)
model.save(fileSegWordDonePath2)
print("Finish")

Finish


# Result

In [ ]:
# -*- coding: utf-8 -*-# -*- coding: utf-8 -*-
import sys
from gensim.models import word2vec 
from gensim import models
from hanziconv import HanziConv

def main():   
    a = input("input:") 
    Simp = HanziConv.toSimplified(a)
    model = models.Word2Vec.load(fileSegWordDonePath2)
    try:
        res = model.most_similar(Simp, topn=10)
        for item in res: 
            print(HanziConv.toTraditional(item[0]) + ':' + str(item[1]))
    except KeyError:
        print(a + " is not in the vocabulary")

if __name__=='__main__':
    main()